In [1]:
import numpy as np

In [2]:
def differential_evolution(func, bounds, Np=None, F=0.8, CR=0.9, Gmax=1000, tol=1e-6, args=()):
    """
    Differential Evolution (DE/rand/1/bin) with improved defaults.
    """
    D = len(bounds)
    if Np is None:
        Np = 15 * D  # adaptive population size

    # Initialize population
    pop = np.random.uniform(
        [b[0] for b in bounds],
        [b[1] for b in bounds],
        (Np, D)
    )
    fobj = np.array([func(ind, *args) for ind in pop])

    best_value_history = []
    best_solution = pop[np.argmin(fobj)]
    best_value = fobj.min()

    for gen in range(Gmax):
        new_pop = np.empty_like(pop)
        for j in range(Np):
            indices = list(range(Np))
            indices.remove(j)
            a, b, c = np.random.choice(indices, 3, replace=False)
            mutant = pop[a] + F * (pop[b] - pop[c])

            # Crossover
            cross_points = np.random.rand(D) < CR
            if not np.any(cross_points):
                cross_points[np.random.randint(0, D)] = True
            trial = np.where(cross_points, mutant, pop[j])

            # Enforce bounds
            for k in range(D):
                trial[k] = np.clip(trial[k], bounds[k][0], bounds[k][1])

            # Selection
            f_trial = func(trial, *args)
            if f_trial < fobj[j]:
                new_pop[j] = trial
                fobj[j] = f_trial
            else:
                new_pop[j] = pop[j]

        pop = new_pop
        gen_best_idx = np.argmin(fobj)
        gen_best_val = fobj[gen_best_idx]
        best_value_history.append(gen_best_val)

        if gen_best_val < best_value:
            best_value = gen_best_val
            best_solution = pop[gen_best_idx].copy()

        # Early stopping if improvement is tiny
        if gen > 50 and abs(best_value_history[-1] - best_value_history[-50]) < tol:
            print(f"Early stopping at generation {gen}")
            break

        print(f"Iteration {gen}: Best value = {best_value:.6f}")

    return best_solution, best_value, best_value_history

In [3]:
# --- Common benchmark functions
def sphere(x):
    return np.sum(x**2)

def rastrigin(x):
    A = 10
    return A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x))

def rosenbrock(x):
    return np.sum(100 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

def ackley(x):
    a = 20
    b = 0.2
    c = 2 * np.pi
    n = len(x)
    s1 = np.sum(x**2)
    s2 = np.sum(np.cos(c * x))
    return -a * np.exp(-b * np.sqrt(s1 / n)) - np.exp(s2 / n) + a + np.e

In [4]:
if __name__ == "__main__":
    # List of test problems
    test_functions = [
        ("Sphere", sphere, [(-5.12, 5.12)] * 5),
        ("Rastrigin", rastrigin, [(-5.12, 5.12)] * 5),
        ("Rosenbrock", rosenbrock, [(-2.048, 2.048)] * 5),
        ("Ackley", ackley, [(-32.768, 32.768)] * 5),
    ]

    for name, func, bounds in test_functions:
        print(f"\n=== Optimizing {name} Function ===")
        best_sol, best_val, history = differential_evolution(
            func, bounds, Np=50, F=0.7, CR=0.9, Gmax=800
        )
        print(f"Best solution: {best_sol}")
        print(f"Best value: {best_val:.6e}")


=== Optimizing Sphere Function ===
Iteration 0: Best value = 5.245775
Iteration 1: Best value = 5.245775
Iteration 2: Best value = 5.245775
Iteration 3: Best value = 2.754414
Iteration 4: Best value = 2.754414
Iteration 5: Best value = 2.754414
Iteration 6: Best value = 2.754414
Iteration 7: Best value = 2.754414
Iteration 8: Best value = 2.754414
Iteration 9: Best value = 1.520315
Iteration 10: Best value = 1.519509
Iteration 11: Best value = 1.451307
Iteration 12: Best value = 1.403334
Iteration 13: Best value = 1.403334
Iteration 14: Best value = 1.251072
Iteration 15: Best value = 0.759659
Iteration 16: Best value = 0.298929
Iteration 17: Best value = 0.298929
Iteration 18: Best value = 0.298929
Iteration 19: Best value = 0.298929
Iteration 20: Best value = 0.298929
Iteration 21: Best value = 0.298929
Iteration 22: Best value = 0.298929
Iteration 23: Best value = 0.245301
Iteration 24: Best value = 0.245301
Iteration 25: Best value = 0.213199
Iteration 26: Best value = 0.213199
It